In [10]:
import matplotlib.pyplot as plt
from ekpy import control
from ekpy.control.instruments import keysight81150a, keysightdsox3024a
from dwm import DWM
import numpy as np

In [2]:
rm = control.ResourceManager()
rm.list_resources()

('GPIB0::7::INSTR',
 'GPIB0::8::INSTR',
 'USB0::0x0957::0x17A6::MY63080078::0::INSTR')

In [3]:
scope = rm.open_resource('GPIB0::7::INSTR')
wavegen = rm.open_resource('GPIB0::8::INSTR')

In [4]:
keysightdsox3024a.idn(scope)
keysight81150a.idn(wavegen)

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [5]:
exp_fe = DWM(wavegen,scope)

# specify run function (if desired)
# options are: run_function aka i havent added a different one


# config data path
Data_folder_path = "C:/Users/geofr/Desktop/Research/Data/FE/DWM/"

exp_fe.config_path(Data_folder_path + r'test')

In [6]:
voltages = ['1'] 
scan_params = {
    'voltage':voltages, #Note overwrites values below so a voltage of 2 is not given in this context
}

run_function_args = {
    'wavegen':wavegen,
    'scope':scope,
    'pulse_width':'1e-3',
    'pulse_delay':'1e-4',
    'voltage':'2',
    'capacitor_area':'4e-8',
    'thickness':'10e-9',
    'permittivity':'30',
}

order = ['voltage',]

In [7]:
exp_fe.n_param_scan(scan_params, run_function_args, order, ntrials=1, plot=False)

done.
